In [1]:
# Importing Libraries/Dependencies
import selenium
import pandas as pd
import time
import requests
from selenium import webdriver
from  bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException

### FLIPKART

In [5]:
# Connecting to the webdriver 
driver = webdriver.Chrome("chromedriver.exe")
driver.maximize_window()        # Maximizing the window

c:\Temp/ipykernel_12220/1597520620.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe")


In [6]:
# Getting the webpage of mentioned url
url = "https://www.flipkart.com/"
driver.get(url)
time.sleep(2)


In [7]:
# Listing down the items that need to be searched
srch_items = ['laptops', 'Phones','smart watches','Monitors']

In [8]:
# Creating empty lists to fetch required data
title = []           # Title of the product
review_text = []     # Text of the review or comments of the particular product
ratings = []         # Rating of the product

In [9]:
# Defining function to scrap the required data of products
def scrap():    
        for i in driver.find_elements(By.XPATH,"//div[@class='t-ZTKy']"):
            review_text.append(i.text)
        for i in driver.find_elements(By.XPATH,"//p[@class='_2-N8zT']"):
            title.append(i.text)
        for i in driver.find_elements(By.XPATH,"//p[@class='_2-N8zT']/../div"):
            ratings.append(i.text)
        return

In [10]:
# Creating empty list to fetch the page URLs of the products
urls=[]

In [11]:
for i in srch_items:
    # Find search bar
    srchBar = driver.find_element(By.XPATH,"//div[@class='_3OO5Xc']/input")
    srchBar.clear()
    srchBar.send_keys(i)
    # Clicking on search button
    driver.find_element(By.XPATH,"//button[@class='L0Z3Pu']").click()
    time.sleep(3)
    page = []
    for i in driver.find_elements(By.XPATH,"//nav[@class='yFHi8N']/a"):
        page.append(i.get_attribute('href'))
    for i in page[0:4]:
        driver.get(i)
        time.sleep(3)
        items = driver.find_elements(By.XPATH,"//a[@class='_1fQZEK']")
        for i in items:
            urls.append(i.get_attribute('href'))

In [12]:
# Checking the length of the url
len(urls)

288

In [13]:
# Fetching the required data from the website
for i in urls:
    driver.get(i)
    time.sleep(2)
    for _ in range(2):
        driver.execute_script("window.scrollBy(0,6000)")
        time.sleep(1)
        # Clicking on all reviews
    try:
        btn=driver.find_element(By.XPATH,"//div[@class='_2c2kV-']/following::a")
        lnk = btn.get_attribute('href')
        driver.get(lnk)
        time.sleep(1)
    except NoSuchElementException:
        pass
    
# Calling the function
    scrap()        
    try:
        n_page=driver.find_elements(By.XPATH,"//nav[@class='yFHi8N']/a")
        np=[]
        for i in n_page:
            np.append(i.get_attribute('href'))
        for i in np[0:18]:
            driver.get(i)
            time.sleep(1)
            scrap()
    except: continue

WebDriverException: Message: disconnected: Unable to receive message from renderer
  (Session info: chrome=106.0.5249.119)
Stacktrace:
Backtrace:
	Ordinal0 [0x005EDF13+2219795]
	Ordinal0 [0x00582841+1779777]
	Ordinal0 [0x0049423D+803389]
	Ordinal0 [0x004871D5+750037]
	Ordinal0 [0x0048687C+747644]
	Ordinal0 [0x0049A28F+828047]
	Ordinal0 [0x004EF521+1176865]
	Ordinal0 [0x004DE616+1107478]
	Ordinal0 [0x004B7F89+950153]
	Ordinal0 [0x004B8F56+954198]
	GetHandleVerifier [0x008E2CB2+3040210]
	GetHandleVerifier [0x008D2BB4+2974420]
	GetHandleVerifier [0x00686A0A+565546]
	GetHandleVerifier [0x00685680+560544]
	Ordinal0 [0x00589A5C+1808988]
	Ordinal0 [0x0058E3A8+1827752]
	Ordinal0 [0x0058E495+1827989]
	Ordinal0 [0x005980A4+1867940]
	BaseThreadInitThunk [0x76FB6739+25]
	RtlGetFullPathName_UEx [0x77CF8FD2+1218]
	RtlGetFullPathName_UEx [0x77CF8F9D+1165]


In [14]:
# Checking the length of the columns
len(ratings), len(review_text), len(title)

(17125, 17125, 17125)

In [15]:
# Creating the dataframe for scraped data from the flipkart website
data = list(zip(title,review_text,ratings))
df2 = pd.DataFrame(data, columns = ["Review_title","Reiew_text","Ratings"])
df2

,Review_title,Reiew_text,Ratings
0,Simply awesome,It's very good to this price.....I really than...,5
1,Great product,Seriously awwsm .. pls go for it without any h...,5
2,Fabulous!,Performance are too good and friendly budget a...,5
3,Nice product,I am happy with this laptop. Good for basic us...,4
4,Don't waste your money,Its my review after used after 5 months.\nIts ...,1
...,...,...,...
17120,Best in the market!,Once again Noise has delivered a really good p...,5
17121,Value-for-money,This is my first smart watch and its design is...,4
17122,Simply awesome,Im Pretty Happy about the purchase. All the ac...,5
17123,Superb till the product working.,Excellent Product from Noise.\nBattery- 9 Days...,4


In [16]:
# Saving the scraped data into csv file
df2.to_csv("Flipkrt_reviews.csv")

### AMAZON

In [17]:
# Connecting to the webdriver 
driver = webdriver.Chrome("chromedriver.exe")
driver.maximize_window()        # Maximizing the window

c:\Temp/ipykernel_12220/1597520620.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe")


In [18]:
# Getting the webpage of mentioned url
url = "https://www.amazon.in/"
driver.get(url)
time.sleep(3)

In [19]:
# Listing down the items that need to be searched
srch_items = ['laptops','phones','headphones','smart watches', 'Professional Cameras', 'Printers', 'Monitors', 'Home theater', 'Router']

In [20]:
# Creating empty lists to fetch required data
title = []           # Title of the product
review_text = []     # Text of the review or comments of the particular product
ratings = []         # Rating of the product

In [29]:
# Fetching the required data from the website
for i in srch_items:
    srchbar = driver.find_element(By.ID,"twotabsearchtextbox")
    srchbar.send_keys(i)
    driver.find_element(By.ID,"nav-search-submit-button").click()  # Clicking on search button
    time.sleep(1)
    item_url = []
    for i in driver.find_elements(By.XPATH,"//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-2']/a"):
        item_url.append(i.get_attribute('href'))
    for i in item_url:
        driver.get(i)
        time.sleep(1)
        for _ in range(2):
            driver.execute_script("window.scrollBy(0,6000)")
            time.sleep(1)
        # Clicking on see all reviews
        try:
            driver.find_element(By.XPATH,"//a[@class='a-link-emphasis a-text-bold']").click()
        except:
            pass
        
        url_ = []
        try:
            two_str = driver.find_element(By.XPATH,"//table[@id='histogramTable']/tbody/tr[4]/td[2]/a")
            url_.append(two_str.get_attribute('href'))
        except: pass
        try:
            three_str = driver.find_element(By.XPATH,"//table[@id='histogramTable']/tbody/tr[3]/td[2]/a")
            url_.append(three_str.get_attribute('href'))
        except: pass
        try:
            one_str = driver.find_element(By.XPATH,"//table[@id='histogramTable']/tbody/tr[5]/td[2]/a")
            url_.append(one_str.get_attribute('href'))
        except: pass
        try:
            fiv_str = driver.find_element(By.XPATH,"//table[@id='histogramTable']/tbody/tr[1]/td[2]/a")
            url_.append(fiv_str.get_attribute('href'))
        except: pass
        try:
            four_str = driver.find_element(By.XPATH,"//table[@id='histogramTable']/tbody/tr[2]/td[2]/a")
            url_.append(four_str.get_attribute('href'))
        except: pass
        
        for i in url_:
            driver.get(i)
            time.sleep(1)
            for i in range(10):
                ids = driver.find_elements(By.XPATH,"//div[@class='a-section a-spacing-none review-views celwidget']/div")
                comment_ids = []
                for i in ids:
                    comment_ids.append(i.get_attribute('id'))

                for x in comment_ids:
                    # Extract user ids from each user on a page
                    try:
                        review_title = driver.find_element(By.XPATH,'//*[@id="' + x +'"]/div/div/div[2]/a[2]/span[1]')
                        title.append(review_title.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        title.append('')

                    try:
                        rating = driver.find_element(By.XPATH,'//*[@id="' + x +'"]/div/div/div[2]/a[1]/i/span[1]')
                        ratings.append(rating.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        ratings.append('')

                    try:
                        text = driver.find_element(By.XPATH,'//*[@id="' + x +'"]/div/div/div[4]/span/span')
                        review_text.append(text.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        review_text.append('')
                try:
                    driver.find_element(By.XPATH,"//ul[@class='a-pagination']/li[2]/a").click()
                    time.sleep(3)
                except : break 

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=106.0.5249.119)
Stacktrace:
Backtrace:
	Ordinal0 [0x005EDF13+2219795]
	Ordinal0 [0x00582841+1779777]
	Ordinal0 [0x0049423D+803389]
	Ordinal0 [0x00496D04+814340]
	Ordinal0 [0x00496BC2+814018]
	Ordinal0 [0x0049755F+816479]
	Ordinal0 [0x004EFC1B+1178651]
	Ordinal0 [0x004DE7FC+1107964]
	Ordinal0 [0x004EF192+1175954]
	Ordinal0 [0x004DE616+1107478]
	Ordinal0 [0x004B7F89+950153]
	Ordinal0 [0x004B8F56+954198]
	GetHandleVerifier [0x008E2CB2+3040210]
	GetHandleVerifier [0x008D2BB4+2974420]
	GetHandleVerifier [0x00686A0A+565546]
	GetHandleVerifier [0x00685680+560544]
	Ordinal0 [0x00589A5C+1808988]
	Ordinal0 [0x0058E3A8+1827752]
	Ordinal0 [0x0058E495+1827989]
	Ordinal0 [0x005980A4+1867940]
	BaseThreadInitThunk [0x76FB6739+25]
	RtlGetFullPathName_UEx [0x77CF8FD2+1218]
	RtlGetFullPathName_UEx [0x77CF8F9D+1165]


In [30]:
len(ratings), len(title), len(review_text)

(7179, 7180, 7179)

In [31]:
#creating a dataframe
data = list(zip(title,review_text,ratings))
df = pd.DataFrame(data, columns = ["Review_title","Reiew_text","Ratings"])
df

,Review_title,Reiew_text,Ratings
0,,,
1,Just OK,Battery back up is very poor. Last 2.5 to 3 ho...,3.0 out of 5 stars
2,Not bad,,3.0 out of 5 stars
3,Received faulty product,Don't Purchase<br><br>Cooling Fan noise issue<...,1.0 out of 5 stars
4,Overheating,Defective design<br>Overheating<br>Vent at the...,1.0 out of 5 stars
...,...,...,...
7174,Good phone with average camera,"Good performance, decent battery life and char...",4.0 out of 5 stars
7175,Heavy weight,"The phone is very heavy by the weight, would b...",4.0 out of 5 stars
7176,Redmi 11T,Camera is not soo good<br>Otherwise it's ok,4.0 out of 5 stars
7177,sound quality,this phone is overall good under 14k but sound...,4.0 out of 5 stars


In [32]:
# Saving the data into csv file
df.to_csv("Amazon_reviews.csv")

In [33]:
# lets load the flipkart reviews file
flpkrt = pd.read_csv("Flipkrt_reviews.csv")
flpkrt

,Unnamed: 0,Review_title,Reiew_text,Ratings
0,0,Simply awesome,It's very good to this price.....I really than...,5
1,1,Great product,Seriously awwsm .. pls go for it without any h...,5
2,2,Fabulous!,Performance are too good and friendly budget a...,5
3,3,Nice product,I am happy with this laptop. Good for basic us...,4
4,4,Don't waste your money,Its my review after used after 5 months.\nIts ...,1
...,...,...,...,...
17120,17120,Best in the market!,Once again Noise has delivered a really good p...,5
17121,17121,Value-for-money,This is my first smart watch and its design is...,4
17122,17122,Simply awesome,Im Pretty Happy about the purchase. All the ac...,5
17123,17123,Superb till the product working.,Excellent Product from Noise.\nBattery- 9 Days...,4


In [34]:
# Concatenating the two datasets 
reviews = pd.concat([df, flpkrt],ignore_index=True)
reviews

,Review_title,Reiew_text,Ratings,Unnamed: 0
0,,,,NaN
1,Just OK,Battery back up is very poor. Last 2.5 to 3 ho...,3.0 out of 5 stars,NaN
2,Not bad,,3.0 out of 5 stars,NaN
3,Received faulty product,Don't Purchase<br><br>Cooling Fan noise issue<...,1.0 out of 5 stars,NaN
4,Overheating,Defective design<br>Overheating<br>Vent at the...,1.0 out of 5 stars,NaN
...,...,...,...,...
24299,Best in the market!,Once again Noise has delivered a really good p...,5,17120.0
24300,Value-for-money,This is my first smart watch and its design is...,4,17121.0
24301,Simply awesome,Im Pretty Happy about the purchase. All the ac...,5,17122.0
24302,Superb till the product working.,Excellent Product from Noise.\nBattery- 9 Days...,4,17123.0


In [35]:
# Dropping unwanted column
reviews.drop(columns = 'Unnamed: 0', inplace=True)

In [36]:
# Displaying the merged dataset
reviews

,Review_title,Reiew_text,Ratings
0,,,
1,Just OK,Battery back up is very poor. Last 2.5 to 3 ho...,3.0 out of 5 stars
2,Not bad,,3.0 out of 5 stars
3,Received faulty product,Don't Purchase<br><br>Cooling Fan noise issue<...,1.0 out of 5 stars
4,Overheating,Defective design<br>Overheating<br>Vent at the...,1.0 out of 5 stars
...,...,...,...
24299,Best in the market!,Once again Noise has delivered a really good p...,5
24300,Value-for-money,This is my first smart watch and its design is...,4
24301,Simply awesome,Im Pretty Happy about the purchase. All the ac...,5
24302,Superb till the product working.,Excellent Product from Noise.\nBattery- 9 Days...,4


In [37]:
# Saving the final data into csv file
reviews.to_csv("reviews&ratings.csv")